# Microsoft Agent Framework सह मानव-इन-द-लूप कार्यप्रवाह

## 🎯 शिकण्याची उद्दिष्टे

या नोटबुकमध्ये, तुम्ही Microsoft Agent Framework च्या `RequestInfoExecutor` वापरून **मानव-इन-द-लूप** कार्यप्रवाह कसा अंमलात आणायचा ते शिकाल. हा शक्तिशाली पॅटर्न AI कार्यप्रवाह थांबवून मानवी इनपुट गोळा करण्यास अनुमती देतो, ज्यामुळे तुमचे एजंट परस्परसंवादी बनतात आणि महत्त्वाच्या निर्णयांवर मानवी नियंत्रण मिळते.

## 🔄 मानव-इन-द-लूप म्हणजे काय?

**मानव-इन-द-लूप (HITL)** हा एक डिझाइन पॅटर्न आहे जिथे AI एजंट पुढे जाण्यापूर्वी मानवी इनपुटची विनंती करण्यासाठी अंमलबजावणी थांबवतात. हे खालील गोष्टींसाठी आवश्यक आहे:

- ✅ **महत्त्वाचे निर्णय** - महत्त्वाचे कृती करण्यापूर्वी मानवी मंजुरी मिळवा
- ✅ **संदिग्ध परिस्थिती** - AI ला शंका असल्यास मानवांना स्पष्ट करण्याची परवानगी द्या
- ✅ **वापरकर्ता प्राधान्ये** - अनेक पर्यायांपैकी निवडण्यासाठी वापरकर्त्यांना विचारा
- ✅ **अनुपालन आणि सुरक्षा** - नियमन केलेल्या ऑपरेशन्ससाठी मानवी देखरेख सुनिश्चित करा
- ✅ **परस्परसंवादी अनुभव** - वापरकर्त्याच्या इनपुटला प्रतिसाद देणारे संवादात्मक एजंट तयार करा

## 🏗️ Microsoft Agent Framework मध्ये हे कसे कार्य करते

फ्रेमवर्क HITL साठी तीन प्रमुख घटक प्रदान करते:

1. **`RequestInfoExecutor`** - एक विशेष कार्यकारी जो कार्यप्रवाह थांबवतो आणि `RequestInfoEvent` उत्सर्जित करतो
2. **`RequestInfoMessage`** - मानवांना पाठवलेल्या टाइप केलेल्या विनंती पेलोडसाठी बेस क्लास
3. **`RequestResponse`** - `request_id` वापरून मानवी प्रतिसादांना मूळ विनंतीशी जोडतो

**कार्यप्रवाह पॅटर्न:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 आमचे उदाहरण: वापरकर्ता पुष्टीकरणासह हॉटेल बुकिंग

आम्ही पर्यायी गंतव्यस्थाने सुचवण्यापूर्वी मानवी पुष्टीकरण जोडून सशर्त कार्यप्रवाहावर आधारित तयार करू:

1. वापरकर्ता गंतव्यस्थानाची विनंती करतो (उदा., "पॅरिस")
2. `availability_agent` तपासतो की खोल्या उपलब्ध आहेत का
3. **जर खोल्या उपलब्ध नसतील** → `confirmation_agent` विचारतो "तुम्हाला पर्याय पाहायचे आहेत का?"
4. कार्यप्रवाह **थांबतो** `RequestInfoExecutor` वापरून
5. **मानव प्रतिसाद देतो** "होय" किंवा "नाही" कन्सोल इनपुटद्वारे
6. `decision_manager` प्रतिसादावर आधारित मार्ग ठरवतो:
   - **होय** → पर्यायी गंतव्यस्थाने दाखवा
   - **नाही** → बुकिंग विनंती रद्द करा
7. अंतिम परिणाम दाखवा

हे एजंटच्या सूचनांवर वापरकर्त्यांना नियंत्रण कसे द्यायचे ते दाखवते!

---

चला सुरुवात करूया! 🚀


## चरण 1: आवश्यक लायब्ररी आयात करा

आम्ही मानक Agent Framework घटकांसह **मानवी सहभागासाठी विशिष्ट वर्ग** आयात करतो:
- `RequestInfoExecutor` - मानवी इनपुटसाठी कार्यप्रवाह थांबवणारा एक्झिक्युटर
- `RequestInfoEvent` - मानवी इनपुटची विनंती केल्यावर उत्सर्जित होणारी घटना
- `RequestInfoMessage` - टाइप केलेल्या विनंती पेलोडसाठी बेस वर्ग
- `RequestResponse` - मानवी प्रतिसादांना विनंत्यांशी जोडतो
- `WorkflowOutputEvent` - कार्यप्रवाह आउटपुट शोधण्यासाठी घटना


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## चरण 2: संरचित आउटपुटसाठी Pydantic मॉडेल्स परिभाषित करा

हे मॉडेल्स एजंट्स परत करणार असलेल्या **स्कीमा** परिभाषित करतात. आम्ही सशर्त कार्यप्रवाहातील सर्व मॉडेल्स ठेवतो आणि खालील गोष्टी जोडतो:

**मानवी सहभागासाठी नवीन:**
- `HumanFeedbackRequest` - `RequestInfoMessage` चे उपवर्ग, जो मानवांना पाठवले जाणारे विनंती पेलोड परिभाषित करतो
  - यात `prompt` (विचारायचा प्रश्न) आणि `destination` (उपलब्ध नसलेल्या शहराबद्दल संदर्भ) समाविष्ट आहे


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## चरण 3: हॉटेल बुकिंग साधन तयार करा

सशर्त कार्यप्रवाहातील समान साधन - गंतव्यस्थानातील खोल्या उपलब्ध आहेत का ते तपासते.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## चरण 4: रूटिंगसाठी अटींच्या फंक्शन्सची व्याख्या करा

आपल्या मानव-इन-द-लूप कार्यप्रवाहासाठी **चार अटींच्या फंक्शन्स** आवश्यक आहेत:

**सशर्त कार्यप्रवाहातून:**
1. `has_availability_condition` - जेव्हा हॉटेल्स उपलब्ध असतात तेव्हा रूटिंग करते
2. `no_availability_condition` - जेव्हा हॉटेल्स उपलब्ध नसतात तेव्हा रूटिंग करते

**मानव-इन-द-लूपसाठी नवीन:**
3. `user_wants_alternatives_condition` - जेव्हा वापरकर्ता पर्यायांना "होय" म्हणतो तेव्हा रूटिंग करते
4. `user_declines_alternatives_condition` - जेव्हा वापरकर्ता पर्यायांना "नाही" म्हणतो तेव्हा रूटिंग करते


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## चरण 5: निर्णय व्यवस्थापक कार्यान्वितकर्ता तयार करा

हा **मानवी सहभागाचा मुख्य भाग आहे**! `DecisionManager` हा एक सानुकूल `Executor` आहे जो:

1. **मानवी अभिप्राय प्राप्त करतो** `RequestResponse` ऑब्जेक्ट्सद्वारे
2. **वापरकर्त्याचा निर्णय प्रक्रिया करतो** (होय/नाही)
3. **कार्यप्रवाह मार्गित करतो** योग्य एजंट्सकडे संदेश पाठवून

मुख्य वैशिष्ट्ये:
- `@handler` डेकोरेटर वापरतो जेणेकरून पद्धती कार्यप्रवाह चरण म्हणून उघड करता येतील
- `RequestResponse[HumanFeedbackRequest, str]` प्राप्त करतो ज्यामध्ये मूळ विनंती आणि वापरकर्त्याचे उत्तर दोन्ही असते
- सोपे "होय" किंवा "नाही" संदेश तयार करतो जे आमच्या अटी कार्ये ट्रिगर करतात


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## चरण 6: सानुकूल डिस्प्ले एक्झिक्युटर तयार करा

सशर्त कार्यप्रवाहातील समान डिस्प्ले एक्झिक्युटर - कार्यप्रवाह आउटपुट म्हणून अंतिम परिणाम प्रदान करतो.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## चरण 7: पर्यावरणीय व्हेरिएबल्स लोड करा

LLM क्लायंट (GitHub Models, Azure OpenAI, किंवा OpenAI) कॉन्फिगर करा.


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## चरण 8: AI एजंट्स आणि एक्झिक्युटर्स तयार करा

आम्ही **सहा वर्कफ्लो घटक** तयार करतो:

**एजंट्स (AgentExecutor मध्ये गुंडाळलेले):**
1. **availability_agent** - टूल वापरून हॉटेलची उपलब्धता तपासतो
2. **confirmation_agent** - 🆕 मानवी पुष्टीकरण विनंती तयार करतो
3. **alternative_agent** - पर्यायी शहरांचे सुचवतो (जेव्हा वापरकर्ता हो म्हणतो)
4. **booking_agent** - बुकिंगसाठी प्रोत्साहन देतो (जेव्हा खोल्या उपलब्ध असतात)
5. **cancellation_agent** - 🆕 रद्द करण्याचा संदेश हाताळतो (जेव्हा वापरकर्ता नाही म्हणतो)

**विशेष एक्झिक्युटर्स:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` जो मानवी इनपुटसाठी वर्कफ्लो थांबवतो
7. **decision_manager** - 🆕 कस्टम एक्झिक्युटर जो मानवी प्रतिसादावर आधारित मार्गक्रमण करतो (वर आधीच परिभाषित केले आहे)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## चरण 9: मानव-इन-द-लूपसह कार्यप्रवाह तयार करा

आता आपण **शर्तीवर आधारित मार्गक्रमण**सह कार्यप्रवाह ग्राफ तयार करतो, ज्यामध्ये मानव-इन-द-लूपचा मार्ग समाविष्ट आहे:

**कार्यप्रवाह संरचना:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**महत्त्वाचे मार्ग:**
- `availability_agent → confirmation_agent` (जेव्हा कोणतेही खोली उपलब्ध नाहीत)
- `confirmation_agent → prepare_human_request` (प्रकार रूपांतरित करा)
- `prepare_human_request → request_info_executor` (मानवासाठी थांबा)
- `request_info_executor → decision_manager` (नेहमी - RequestResponse प्रदान करते)
- `decision_manager → alternative_agent` (जेव्हा वापरकर्ता "होय" म्हणतो)
- `decision_manager → cancellation_agent` (जेव्हा वापरकर्ता "नाही" म्हणतो)
- `availability_agent → booking_agent` (जेव्हा खोली उपलब्ध असते)
- सर्व मार्ग `display_result` येथे संपतात


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## चरण 10: चाचणी प्रकरण 1 चालवा - उपलब्धता नसलेले शहर (पॅरिस मानवी पुष्टीसह)

ही चाचणी **मानवी सहभागाचा पूर्ण चक्र** दाखवते:

1. पॅरिसमध्ये हॉटेलची विनंती करा
2. availability_agent तपासतो → कोणतेही खोली उपलब्ध नाही
3. confirmation_agent मानवी-सामोरे प्रश्न तयार करतो
4. request_info_executor **कार्यप्रवाह थांबवतो** आणि `RequestInfoEvent` उत्सर्जित करतो
5. **अर्ज इव्हेंट शोधतो आणि कन्सोलमध्ये वापरकर्त्याला सूचना देतो**
6. वापरकर्ता "होय" किंवा "नाही" टाइप करतो
7. अर्ज `send_responses_streaming()` द्वारे प्रतिसाद पाठवतो
8. decision_manager प्रतिसादावर आधारित मार्ग ठरवतो
9. अंतिम निकाल प्रदर्शित होतो

**महत्त्वाचा नमुना:**
- पहिल्या पुनरावृत्तीसाठी `workflow.run_stream()` वापरा
- पुढील पुनरावृत्त्यांसाठी `workflow.send_responses_streaming(pending_responses)` वापरा
- मानवी इनपुट आवश्यक असल्यास शोधण्यासाठी `RequestInfoEvent` ऐका
- अंतिम निकाल कॅप्चर करण्यासाठी `WorkflowOutputEvent` ऐका


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## चरण 11: चाचणी प्रकरण 2 चालवा - उपलब्धता असलेले शहर (स्टॉकहोम - मानवी इनपुट आवश्यक नाही)

ही चाचणी दर्शवते की **थेट मार्ग** जेव्हा खोल्या उपलब्ध असतात:

1. स्टॉकहोममध्ये हॉटेलची विनंती करा
2. availability_agent तपासतो → खोल्या उपलब्ध ✅
3. booking_agent बुकिंग सुचवतो
4. display_result पुष्टी दाखवतो
5. **मानवी इनपुट आवश्यक नाही!**

जेव्हा खोल्या उपलब्ध असतात तेव्हा कार्यप्रवाह पूर्णपणे मानवी-इन-द-लूप मार्ग टाळतो.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## मुख्य मुद्दे आणि मानव-इन-द-लूप सर्वोत्तम पद्धती

### ✅ तुम्ही काय शिकलात:

#### 1. **RequestInfoExecutor पॅटर्न**
मायक्रोसॉफ्ट एजंट फ्रेमवर्कमधील मानव-इन-द-लूप पॅटर्न तीन मुख्य घटकांचा वापर करतो:
- `RequestInfoExecutor` - वर्कफ्लो थांबवतो आणि इव्हेंट्स उत्सर्जित करतो
- `RequestInfoMessage` - टाइप केलेल्या विनंती पेलोडसाठी बेस क्लास (याला सबक्लास करा!)
- `RequestResponse` - मानवी प्रतिसादांना मूळ विनंतीशी जोडतो

**महत्त्वाची समज:**
- `RequestInfoExecutor` स्वतः इनपुट गोळा करत नाही - तो फक्त वर्कफ्लो थांबवतो
- तुमच्या अॅप्लिकेशन कोडला `RequestInfoEvent` ऐकावे लागेल आणि इनपुट गोळा करावा लागेल
- तुम्हाला `send_responses_streaming()` कॉल करावे लागेल ज्यामध्ये `request_id` ते वापरकर्त्याच्या उत्तराचे मॅपिंग असलेला डिक्शनरी असेल

#### 2. **स्ट्रीमिंग एक्झिक्युशन पॅटर्न**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **इव्हेंट-ड्रिव्हन आर्किटेक्चर**
वर्कफ्लो नियंत्रित करण्यासाठी विशिष्ट इव्हेंट्स ऐका:
- `RequestInfoEvent` - मानवी इनपुट आवश्यक आहे (वर्कफ्लो थांबलेला आहे)
- `WorkflowOutputEvent` - अंतिम निकाल उपलब्ध आहे (वर्कफ्लो पूर्ण)
- `WorkflowStatusEvent` - स्थिती बदल (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, इ.)

#### 4. **@handler सह कस्टम एक्झिक्युटर्स**
`DecisionManager` हे दाखवते की एक्झिक्युटर्स कसे तयार करायचे:
- `@handler` डेकोरेटर वापरून पद्धती वर्कफ्लो स्टेप्स म्हणून उघड करा
- टाइप केलेले मेसेजेस प्राप्त करा (उदा., `RequestResponse[HumanFeedbackRequest, str]`)
- वर्कफ्लो इतर एक्झिक्युटर्सकडे मेसेजेस पाठवून रूट करा
- `WorkflowContext` च्या माध्यमातून संदर्भावर प्रवेश करा

#### 5. **मानवी निर्णयांसह सशर्त रूटिंग**
तुम्ही मानवी प्रतिसादांचे मूल्यांकन करणारे अटी फंक्शन्स तयार करू शकता:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 वास्तविक-जगातील अनुप्रयोग:

1. **मंजुरी वर्कफ्लो**
   - खर्च अहवाल प्रक्रिया करण्यापूर्वी व्यवस्थापकाची मंजुरी मिळवा
   - स्वयंचलित ईमेल पाठवण्यापूर्वी मानवी पुनरावलोकन आवश्यक आहे
   - उच्च-मूल्य व्यवहार अंमलात आणण्यापूर्वी पुष्टी करा

2. **सामग्री मॉडरेशन**
   - संशयास्पद सामग्री मानवी पुनरावलोकनासाठी फ्लॅग करा
   - सीमावर्ती प्रकरणांवर अंतिम निर्णय घेण्यासाठी मॉडरेटरला विचारा
   - जेव्हा एआयचा आत्मविश्वास कमी असेल तेव्हा मानवांकडे पाठवा

3. **ग्राहक सेवा**
   - एआयला स्वयंचलितपणे नियमित प्रश्न हाताळू द्या
   - जटिल समस्यांना मानवी एजंटकडे पाठवा
   - ग्राहकाला विचारले की त्यांना मानवाशी बोलायचे आहे का

4. **डेटा प्रक्रिया**
   - अस्पष्ट डेटा नोंदी सोडवण्यासाठी मानवांना विचारा
   - अस्पष्ट दस्तऐवजांवरील एआयच्या व्याख्या पुष्टी करा
   - अनेक वैध व्याख्यांपैकी वापरकर्त्यांना निवड करू द्या

5. **सुरक्षितता-महत्त्वाचे प्रणाली**
   - अपरिवर्तनीय क्रिया करण्यापूर्वी मानवी पुष्टी आवश्यक आहे
   - संवेदनशील डेटावर प्रवेश करण्यापूर्वी मंजुरी मिळवा
   - नियमन केलेल्या उद्योगांमध्ये (आरोग्यसेवा, वित्त) निर्णयांची पुष्टी करा

6. **इंटरएक्टिव्ह एजंट्स**
   - फॉलो-अप प्रश्न विचारणारे संभाषणात्मक बॉट्स तयार करा
   - वापरकर्त्यांना जटिल प्रक्रियांमधून मार्गदर्शन करणारे विजार्ड तयार करा
   - मानवी सहकार्याने पायरी-पायरीने काम करणारे एजंट्स डिझाइन करा

### 🔄 तुलना: मानवी-इन-द-लूप असलेले आणि नसलेले वर्कफ्लो

| वैशिष्ट्य | सशर्त वर्कफ्लो | मानवी-इन-द-लूप वर्कफ्लो |
|-----------|----------------|-------------------------|
| **अंमलबजावणी** | एकल `workflow.run()` | `run_stream()` + `send_responses_streaming()` सह लूप |
| **वापरकर्ता इनपुट** | नाही (पूर्णपणे स्वयंचलित) | `input()` किंवा UI द्वारे संवादात्मक प्रॉम्प्ट |
| **घटक** | एजंट्स + एक्झिक्युटर्स | + RequestInfoExecutor + DecisionManager |
| **इव्हेंट्स** | फक्त AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, इ. |
| **थांबवणे** | थांबवणे नाही | RequestInfoExecutor वर वर्कफ्लो थांबतो |
| **मानवी नियंत्रण** | मानवी नियंत्रण नाही | मानव महत्त्वाचे निर्णय घेतात |
| **वापर प्रकरण** | स्वयंचलन | सहकार्य आणि देखरेख |

### 🚀 प्रगत पद्धती:

#### एकाधिक मानवी निर्णय बिंदू
तुमच्या वर्कफ्लोमध्ये अनेक `RequestInfoExecutor` नोड्स असू शकतात:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### टाइमआउट हाताळणी
मानवी प्रतिसादांसाठी टाइमआउट्स अंमलात आणा:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### समृद्ध UI एकत्रीकरण
`input()` च्या ऐवजी, वेब UI, Slack, Teams, इत्यादींसह एकत्रित करा:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### सशर्त मानवी-इन-द-लूप
विशिष्ट परिस्थितीतच मानवी इनपुटची मागणी करा:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ सर्वोत्तम पद्धती:

1. **नेहमी RequestInfoMessage सबक्लास करा**
   - प्रकार सुरक्षा आणि सत्यापन प्रदान करते
   - UI रेंडरिंगसाठी समृद्ध संदर्भ सक्षम करते
   - प्रत्येक विनंती प्रकाराचा हेतू स्पष्ट करते

2. **वर्णनात्मक प्रॉम्प्ट्स वापरा**
   - तुम्ही काय विचारत आहात याबद्दल संदर्भ समाविष्ट करा
   - प्रत्येक निवडीचे परिणाम स्पष्ट करा
   - प्रश्न सोपे आणि स्पष्ट ठेवा

3. **अनपेक्षित इनपुट हाताळा**
   - वापरकर्ता प्रतिसादांची पडताळणी करा
   - अवैध इनपुटसाठी डीफॉल्ट्स प्रदान करा
   - स्पष्ट त्रुटी संदेश द्या

4. **विनंती आयडी ट्रॅक करा**
   - request_id आणि प्रतिसादांमधील संबंध वापरा
   - स्थिती मॅन्युअली व्यवस्थापित करण्याचा प्रयत्न करू नका

5. **नॉन-ब्लॉकिंगसाठी डिझाइन करा**
   - इनपुटची वाट पाहत थ्रेड्स ब्लॉक करू नका
   - संपूर्णपणे असिंक्रोनस पद्धती वापरा
   - समांतर वर्कफ्लो उदाहरणांना समर्थन द्या

### 📚 संबंधित संकल्पना:

- **एजंट मिडलवेअर** - एजंट कॉल्स इंटरसेप्ट करा (मागील नोटबुक)
- **वर्कफ्लो स्टेट मॅनेजमेंट** - रन दरम्यान वर्कफ्लो स्थिती टिकवून ठेवा
- **मल्टी-एजंट सहकार्य** - मानव-इन-द-लूप एजंट टीम्ससह एकत्र करा
- **इव्हेंट-ड्रिव्हन आर्किटेक्चर** - इव्हेंट्ससह प्रतिक्रियाशील प्रणाली तयार करा

---

### 🎓 अभिनंदन!

तुम्ही मायक्रोसॉफ्ट एजंट फ्रेमवर्कसह मानव-इन-द-लूप वर्कफ्लोमध्ये प्राविण्य मिळवले आहे! तुम्हाला आता माहित आहे की:
- ✅ वर्कफ्लो थांबवून मानवी इनपुट कसे गोळा करायचे
- ✅ RequestInfoExecutor आणि RequestInfoMessage कसे वापरायचे
- ✅ इव्हेंट्ससह स्ट्रीमिंग एक्झिक्युटन कसे हाताळायचे
- ✅ @handler सह कस्टम एक्झिक्युटर्स कसे तयार करायचे
- ✅ मानवी निर्णयांवर आधारित वर्कफ्लो कसे रूट करायचे
- ✅ संवादात्मक एआय एजंट्स तयार करा जे मानवांसोबत सहकार्य करतात

**विश्वसनीय, नियंत्रित एआय प्रणाली तयार करण्यासाठी हा एक महत्त्वाचा पॅटर्न आहे!** 🚀



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरे त्रुटी किंवा अचूकतेच्या अभावाने युक्त असू शकतात. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून उद्भवलेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
